In [3]:
import pandas as pd
import numpy as np

In [3]:
dat = pd.read_csv('./parsed_data/data_parsed/train/Nashville')

In [4]:
(dat['review_scores_rating'] < 4.4).mean()

0.05345129741647352

In [5]:
dat = pd.read_json('./parsed_data/data_stats/train.json')

In [6]:
import os
folder = "./bin0"
tar_folder = './parsed_data/data_parsed/train/'
files = os.listdir(tar_folder)
for file in files:
    dat = pd.read_csv(tar_folder + file)
    tmp = dat.drop(columns=['amenities', 'property_type'])
    # cut bedrooms into 0/1 bedroom, 2 bedrooms, 3 bedrooms, 4+ bedrooms
    #tmp['bedrooms'] = pd.cut(tmp['bedrooms'], bins=[-1, 1, 2, 3, 100], labels=["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"])
    tmp['bedrooms'] = pd.cut(tmp['bedrooms'], bins=[0, 1, 2, 3, 10000], labels=["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"])
    for unique in tmp['bedrooms'].unique():
        print(unique, (tmp['bedrooms'] == unique).mean())
    break
    

nan 0.0
0/1 bedroom 0.29443521594684385
3 bedrooms 0.10465116279069768
2 bedrooms 0.13974252491694353
4+ bedrooms 0.05170265780730897


In [7]:
files
original_name = ['Rochester','Newark','Columbus','BrowardCounty','SanDiego','Cambridge','SantaClaraCounty','ClarkCounty','Dallas','LosAngeles','Salem','SanMateoCounty','Denver','JerseyCity','FortWorth','Oakland','Nashville']
map_name = ['Rochester, NY', 'Newark, NJ', 'Columbus, OH', 'Broward County, FL', 'San Diego, CA', 'Cambridge, MA', 'Santa Clara County, CA', 'Clark County, NV', 'Dallas, TX', 'Los Angeles, CA', 'Salem, OR', 'San Mateo County, CA', 'Denver, CO', 'Jersey City, NJ', 'Fort Worth, TX', 'Oakland, CA', 'Nashville, TN']

map = {}
for i in range(len(original_name)):
    map[original_name[i]] = map_name[i]

In [8]:
import json
dic = {}
vars = []
br = {"Name": "Number of Bedrooms", "Value": ["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"]}
vars.append(br)
bath = {"Name": "Number of Bathrooms", "Value": ["0/1 bathroom", "2 bathrooms", "3+ bathrooms"]}
vars.append(bath)
host_since = {"Name": "Host Experience", "Value": ["<1 year", "1-5 years", "5-10 years", "10+ years"]}
vars.append(host_since)
superhost = {"Name": "Superhost", "Value": ["Yes", "No"]}
vars.append(superhost)
rt = {"Name": "Room Type", "Value": ["Entire home/apt", "Private room", "Shared/Hotel room"]}
vars.append(rt)
beds = {"Name": "Total Beds", "Value": ["0/1 bed", "2 beds", "3 beds", "4+ beds"]}
vars.append(beds)
review = {"Name": "Review Score", "Value": ["<4.3", "4.3-4.8", "4.9+"]}
vars.append(review)
month  = {"Name": "Month", "Value": ["Jan-Apr", "May-Aug", "Sep-Dec"]}
vars.append(month)
acc = {"Name": "Max Accommodates", "Value": ["1", "2", "3", "4", "4+"]}
vars.append(acc)
price = {"Name": "Price", "Value": ["<$50", "$51-$100", "$101-$200", "$201-$500", "$500+"]}
vars.append(price)
dic["Variables"] = vars



In [17]:
import os
import json
import random
random.seed(0)


folder = "./Dataset/bin5_newname"
tar_folder = './parsed_data/data_parsed/train/'
files = os.listdir(tar_folder)
questions = []
for i, file in enumerate(files):

    dat = pd.read_csv(tar_folder + file)
    tmp = dat.drop(columns=['amenities', 'property_type', 'id', 'source'])

    # get nan ratio in bedrooms
    # cut bedrooms into 0/1 bedroom, 2 bedrooms, 3 bedrooms, 4+ bedrooms
    tmp['Number of Bedrooms'] = pd.cut(tmp['bedrooms'], bins=[-1, 1, 2, 3, 100], labels=["studio or 1 bedroom", "2 bedrooms", "3 bedrooms", "4 or more bedrooms"])

    #tmp['bedrooms'] = pd.cut(tmp['bedrooms'], bins=[0, 1, 2, 3, 10000], labels=["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"])
    tmp['Number of Bathrooms'] = pd.cut(tmp['bathrooms'], bins=[-1, 1, 2, 100], labels=["shared or single bathroom", "2 bathrooms", "3 or more bathrooms"])
    tmp['Host Experience'] = pd.cut(tmp['host_since'], bins=[-1, 365, 365*5, 365 * 10, 9999999999], labels=["under 1 year", "1 to 5 years", "6 to 10 years", "more than 10 years"])
    #tmp['host_is_superhost'] = tmp['host_is_superhost'].astype(str)
    # put host is superhost as 'yes' or 'no', nan as nan

    def map_values(val):
        if pd.isna(val):
            return np.nan
        elif val:
            return 'Yes'
        else:
            return 'No'

    mapping = {True: 'Yes', False: 'No'}

    # Create a new column with the mapped values
    tmp['Superhost'] = tmp['host_is_superhost'].apply(map_values)


    tmp['Room Type'] = tmp['room_type'].apply(lambda x: x if x in ['entire home or apartment', 'private room'] else 'shared or hotel room')

    # drop the zeros in beds
    tmp = tmp[tmp['beds'] != 0]

    tmp['Total Beds'] = pd.cut(tmp['beds'], bins=[-1, 1, 2, 3, 100], labels=["1 bed", "2 beds", "3 beds", "4 or more beds"])
    print((tmp['beds'] == 0).mean(), file, len(tmp))
    tmp['Review Score'] = pd.cut(tmp['review_scores_rating'], bins=[-1, 4.4, 4.9, 5.1], labels=["less than 4.4", "4.5 to 4.8", "at least 4.9"])
    tmp['Price'] = pd.cut(tmp['price'], bins=[-1, 50.9999999, 100.9999999, 200.9999999, 500.9999999, 9999999], labels=["under $50", "$51 to $100", "$101 to $200", "$201 to $500", "at least $501"])
    tmp['Max Accommodates'] = pd.cut(tmp['accommodates'], bins=[-1, 1, 2, 3, 100], labels=["1", "2", "3", "4 or more"])
    #for unique in tmp['accommodates'].unique():
    #    print(unique, (tmp['accommodates'] == unique).mean())
    #break
    # drop the original columns
    tmp = tmp.drop(columns=['bedrooms', 'bathrooms', 'host_since', 'host_is_superhost', 'room_type', 'beds', 'review_scores_rating', 'price', 'month', 'accommodates'])

    #for i in tmp['Review Score'].unique():
    #    print(i, (tmp['Review Score'] == i).mean())


    # save back to csv 
    # make directory
    if not os.path.exists(folder + '/' + file):
        os.makedirs(folder + '/' + file)
    tmp.to_csv(folder + '/' + file + '/' + file + '.csv')
    #print(folder + '/' + file + '/' + file + '.csv')
    # save json
    #with open(folder + '/' + file + '/' + file + '.json', 'w') as f:
    #    json.dump(json_rec, f)
    #with open(folder + '/' + file + '/' + file + '_q.json', 'w') as f:
    #    json.dump(json_q, f)
    #questions.append(folder + '/' + file + '/' + file + '_q.json')
    

0.0 Rochester 4710
0.0 Newark 4890
0.0 Columbus 7566
0.0 BrowardCounty 53873
0.0 SanDiego 48661
0.0 Cambridge 3245
0.0 SantaClaraCounty 20433
0.0 ClarkCounty 62321
0.0 Dallas 58019
0.0 LosAngeles 178951
0.0 Salem 961
0.0 SanMateoCounty 13737
0.0 Denver 20566
0.0 JerseyCity 5708
0.0 FortWorth 7180
0.0 Oakland 10217
0.0 Nashville 35218


In [12]:
import os
import json
import random
random.seed(0)

folder = "./Dataset/bin0"
tar_folder = './parsed_data/data_parsed/train/'
files = os.listdir(tar_folder)
questions = []
for i, file in enumerate(files):

    dat = pd.read_csv(tar_folder + file)
    tmp = dat.drop(columns=['amenities', 'property_type', 'id', 'source'])
    # get nan ratio in bedrooms
    # cut bedrooms into 0/1 bedroom, 2 bedrooms, 3 bedrooms, 4+ bedrooms
    tmp['bedrooms'] = pd.cut(tmp['bedrooms'], bins=[-1, 1, 2, 3, 100], labels=["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"])

    #tmp['bedrooms'] = pd.cut(tmp['bedrooms'], bins=[0, 1, 2, 3, 10000], labels=["0/1 bedroom", "2 bedrooms", "3 bedrooms", "4+ bedrooms"])
    tmp['bathrooms'] = pd.cut(tmp['bathrooms'], bins=[-1, 1, 2, 100], labels=["0/1 bathroom", "2 bathrooms", "3+ bathrooms"])
    tmp['host_since'] = pd.cut(tmp['host_since'], bins=[-1, 365, 365*5, 365 * 10, 9999999999], labels=["<1 year", "1-5 years", "5-10 years", "10+ years"])
    #tmp['host_is_superhost'] = tmp['host_is_superhost'].astype(str)
    # put host is superhost as 'yes' or 'no', nan as nan

    def map_values(val):
        if pd.isna(val):
            return np.nan
        elif val:
            return 'Yes'
        else:
            return 'No'

    mapping = {True: 'Yes', False: 'No'}

    # Create a new column with the mapped values
    tmp['host_is_superhost'] = tmp['host_is_superhost'].apply(map_values)


    tmp['room_type'] = tmp['room_type'].apply(lambda x: x if x in ['Entire home/apt', 'Private room'] else 'Shared/Hotel room')
    tmp['beds'] = pd.cut(tmp['beds'], bins=[-1, 1, 2, 3, 100], labels=["0/1 bed", "2 beds", "3 beds", "4+ beds"])
    tmp['review_scores_rating'] = pd.cut(tmp['review_scores_rating'], bins=[-1, 4.3, 4.9, 5.1], labels=["<4.3", "4.3-4.8", "4.9+"])
    tmp['price'] = pd.cut(tmp['price'], bins=[-1, 50, 100, 200, 500, 9999999], labels=["<$50", "$51-$100", "$101-$200", "$201-$500", "$500+"])
    tmp['month'] = pd.cut(tmp['month'], bins=[0, 4, 8, 12], labels=["Jan-Apr", "May-Aug", "Sep-Dec"])
    tmp['accommodates'] = pd.cut(tmp['accommodates'], bins=[-1, 1, 2, 3, 4, 100], labels=["1", "2", "3", "4", "4+"])
    #for unique in tmp['accommodates'].unique():
    #    print(unique, (tmp['accommodates'] == unique).mean())
    #break


    # for each column, make a json to record the possible values and probability
    json_rec = {}
    json_rec["Variables"] = []
    for col in tmp.columns:
        if col == 'price':
            continue
        js = {}
        js['Name'] = col
        js['Value'] = tmp[col].unique().tolist()
        js['Probability'] = []
        for unique in js['Value']:
            js['Probability'].append((tmp[col] == unique).mean())
        json_rec["Variables"].append(js)
    
    # Generate a question json file
    json_q = {}
    json_q['Target'] = {}
    json_q['Target']['Name'] = 'price'
    
    # get one random variable and value, except for price
    random_col = random.choice(tmp.columns)
    while random_col == 'price':
        random_col = random.choice(tmp.columns)
    # drop nan
    uni = tmp[random_col].dropna().unique().tolist()
    uni = [x for x in uni if str(x) != 'nan']
    #print(uni)
    random_val = random.choice(uni)
    #print(random_val)

    json_q['Condition'] = []
    rand = {}
    rand['Name'] = random_col
    rand['Value'] = random_val
    json_q['Condition'].append(rand)
    loc = {}
    loc['Name'] = 'City'
    loc['Value'] = map[file]
    json_q['Condition'].append(loc)
    

    # save back to csv 
    # make directory
    if not os.path.exists(folder + '/' + file):
        os.makedirs(folder + '/' + file)
    tmp.to_csv(folder + '/' + file + '/' + file + '.csv')
    print(folder + '/' + file + '/' + file + '.csv')
    # save json
    #with open(folder + '/' + file + '/' + file + '.json', 'w') as f:
    #    json.dump(json_rec, f)
    #with open(folder + '/' + file + '/' + file + '_q.json', 'w') as f:
    #    json.dump(json_q, f)
    #questions.append(folder + '/' + file + '/' + file + '_q.json')
    

Rochester 4816
./Dataset/bin0/Rochester/Rochester.csv
Newark 4890
./Dataset/bin0/Newark/Newark.csv
Columbus 7566
./Dataset/bin0/Columbus/Columbus.csv
BrowardCounty 53873
./Dataset/bin0/BrowardCounty/BrowardCounty.csv
SanDiego 49013
./Dataset/bin0/SanDiego/SanDiego.csv
Cambridge 3245
./Dataset/bin0/Cambridge/Cambridge.csv
SantaClaraCounty 20433
./Dataset/bin0/SantaClaraCounty/SantaClaraCounty.csv
ClarkCounty 62827
./Dataset/bin0/ClarkCounty/ClarkCounty.csv
Dallas 59509
./Dataset/bin0/Dallas/Dallas.csv
LosAngeles 180219
./Dataset/bin0/LosAngeles/LosAngeles.csv
Salem 961
./Dataset/bin0/Salem/Salem.csv
SanMateoCounty 13832
./Dataset/bin0/SanMateoCounty/SanMateoCounty.csv
Denver 20762
./Dataset/bin0/Denver/Denver.csv
JerseyCity 5799
./Dataset/bin0/JerseyCity/JerseyCity.csv
FortWorth 7268
./Dataset/bin0/FortWorth/FortWorth.csv
Oakland 10332
./Dataset/bin0/Oakland/Oakland.csv
Nashville 35378
./Dataset/bin0/Nashville/Nashville.csv


In [44]:
cs = pd.read_csv('./bin0/Cambridge/Cambridge.csv')

In [45]:
cs.columns

Index(['Unnamed: 0', 'month', 'host_since', 'host_is_superhost', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price',
       'review_scores_rating'],
      dtype='object')

In [47]:
cs['host_is_superhost'].describe()

count     3245
unique       2
top         No
freq      2108
Name: host_is_superhost, dtype: object

In [66]:
example = json.load(open('./example_files/example_question.json'))

# separate 'Text' and the rest of the entries
text = example['Text']
example.pop('Text')
js = json.dumps(example)

prompt = []

system = "Translate the provided JSON data into a question sentence in English. Follow the provided example. Include keyword Airbnb."

user_1 = js
assistant_1 = text

prompt.append({'role': 'system', 'content': system})
prompt.append({'role': 'user', 'content': user_1})
prompt.append({'role': 'assistant', 'content': assistant_1})

In [67]:
prompt

[{'role': 'system',
  'content': 'Translate the provided JSON data into a question sentence in English. Follow the provided example. Include keyword Airbnb.'},
 {'role': 'user',
  'content': '{"Target": [{"Name": "price", "Type": "float"}], "Condition": [{"Name": "room_type", "Type": "str", "Value": ["Private room"]}, {"Name": "beds", "Type": "int", "Value": ["1", "2", "3"]}]}'},
 {'role': 'assistant',
  'content': 'What is the nightly price of an Airbnb listing that is a private room in a property with at most 3 beds?'}]

In [38]:
from utils import *
trans = question_translate(prompt)
for q in questions:
    print(trans.put_new(q))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [196]:
for q in questions[7:8]:
    print(json.load(open(q)))

{'Target': {'Name': 'price'}, 'Condition': [{'Name': 'review_scores_rating', 'Value': nan}, {'Name': 'City', 'Value': 'Clark County, NV'}], 'Text': 'What is the price of Airbnb listings in Clark County, NV that have no review scores available?'}
